In [ ]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import time
from scipy.stats import linregress
import json
import requests

# Import API key
from api_keys import weather_api_key

# Incorporated citipy to determine city based on latitude and longitude
from citipy import citipy

# Output File (CSV)
output_data_file = "output_data/cities.csv"

# Range of latitudes and longitudes
lat_range = (-90, 90)
lng_range = (-180, 180)

In [ ]:
# List for holding lat_lngs and cities
lat_lngs = []
cities = []

# Create a set of random lat and lng combinations
lats = np.random.uniform(lat_range[0], lat_range[1], size=1500)
lngs = np.random.uniform(lng_range[0], lng_range[1], size=1500)
lat_lngs = zip(lats, lngs)

# Identify nearest city for each lat, lng combination
for lat_lng in lat_lngs:
    city = citipy.nearest_city(lat_lng[0], lat_lng[1]).city_name
    
    # If the city is unique, then add it to a our cities list
    if city not in cities:
        cities.append(city)

# Print the city count to confirm sufficient count
len(cities)

In [ ]:
#Perform API calls
# Save config information
url = "http://api.openweathermap.org/data/2.5/weather?"
units = "metric"

# Build query URL
query_url = f"{url}appid={weather_api_key}&units={units}&q="

# Get weather data
weather_response = requests.get(query_url)
weather_json = weather_response.json()
print(weather_json)

In [ ]:
#Use a for loop to go through cities
#Use try and except for going through cities
# query_url = url + "appid=" + weather_api_key + "units" + "&q="
query_url = f"{url}appid={weather_api_key}&units={units}&q="

#either make a dictionary to store key(city) and its values
lat = []
long =[]
max_temp = []
humidity = []
cloudiness = []
wind = []
country = []
date = []

city_name = []

for index,city in enumerate(cities):
    #try and except error
    try:
        response = requests.get(query_url + city).json()
        lat.append(response["coord"]["lat"])
        long.append(response["coord"]["lon"])
        max_temp.append(response["main"]["temp_max"])
        humidity.append(response["main"]["humidity"])
        cloudiness.append(response["clouds"]["all"])
        wind.append(response["wind"]["speed"])
        country.append(response["sys"]["country"])
        date.append(response["dt"])
        city_name.append(city)
        print("City number is " + str(index) + " & city name is " + city)
    except:
        print("City not found")

In [ ]:
#Create DataFrame
weather_df = pd.DataFrame({"City": city_name, "Lat": lat, "Lng": long,
                           "Max Temp": max_temp, "Humidity": humidity,
                           "Cloudiness": cloudiness, "Wind Speed": wind,
                           "Country": country, "Date": date})
weather_df

In [ ]:
#Check which cities have humidity > 100%
humidity_df = weather_df.loc[weather_df["Humidity"] > 100]
humidity_df

There are no cities with humidity > 100%.

In [ ]:
#Export DataFrame to CSV
weather_df.to_csv("output_data/cities.csv")

# Latitude vs. Temperature Plot

In [ ]:
x = lat
y = max_temp

plt.scatter(lat, max_temp, marker="o", facecolors="lightblue", edgecolors="black")

#Add title and other labels
plt.title("City Latitude vs. Max Temperature (07/29/2020)")
plt.xlabel("Latitude")
plt.ylabel("Max Temperature (F)")
plt.grid()
plt.savefig("output_data/Lat_Temp_Plot.png")
plt.show()

The Latitude vs. Temperature plot shows...

# Latitude vs. Humidity Plot

In [ ]:
x = lat
y = humidity

plt.scatter(lat, humidity, marker="o", facecolors="lightblue", edgecolors="black")

#Add title and other labels
plt.title("City Latitude vs. humidity (07/29/2020)")
plt.xlabel("Latitude")
plt.ylabel("Humidity (%)")
plt.grid()
plt.savefig("output_data/Lat_Humidity_Plot.png")
plt.show()

The Latitude vs. Humidity plot shows ...

# Latitude vs. Cloudiness Plot

In [ ]:
x = lat
y = cloudiness

plt.scatter(lat, cloudiness, marker="o", facecolors="lightblue", edgecolors="black")

#Add title and other labels
plt.title("City Latitude vs. Cloudiness (07/29/2020)")
plt.xlabel("Latitude")
plt.ylabel("Cloudiness (%)")
plt.grid()
plt.savefig("output_data/Lat_Cloudiness_Plot.png")
plt.show()

The Latitude vs. Cloudiness plot shows...

# Latitude vs. Wind Speed Plot

In [ ]:
x = lat
y = wind

plt.scatter(lat, wind, marker="o", facecolors="lightblue", edgecolors="black")

#Add title and other labels
plt.title("City Latitude vs. Wind Speed (07/29/2020)")
plt.xlabel("Latitude")
plt.ylabel("Wind Speed (mph)")
plt.grid()
plt.savefig("output_data/Lat_Wind_Plot.png")
plt.show()

The Latitude vs. Wind Speed plot shows...

# LINEAR REGRESSIONS

In [ ]:
#Create Northern Hemisphere DataFrames
northern_df = weather_df.loc[weather_df["Lat"] >= 0]
northern_df

In [ ]:
#Create Southern Hemisphere DataFrames
southern_df = weather_df.loc[weather_df["Lat"] <= 0]
southern_df

# Northern Hemisphere - Max Temp vs. Latitude Linear Regression

In [ ]:
x_values = northern_df["Lat"]
y_values = northern_df["Max Temp"]

(slope, intercept, rvalue, pvalue, stderr) = linregress(x_values, y_values)
regress_values = x_values * slope + intercept
line_eq = "y = " + str(round(slope,2)) + "x + " + str(round(intercept,2))

plt.scatter(x_values, y_values, marker="o", facecolors="lightblue", edgecolors="black")

plt.plot(x_values,regress_values,"r-")

plt.annotate(line_eq,(0,280),fontsize=15,color="red") 

#Add title and other labels
plt.title("Northern Hemisphere: City Latitude vs. Max Temperature (07/29/2020)")
plt.xlabel("Latitude")
plt.ylabel("Max Temperature (F)")

# Print out the r-squared value along with the plot.
print(f"The r-squared is: {rvalue**2}")

# Show and save plot
plt.savefig("output_data/North_Lat_Temp_Plot.png")
plt.show()

# Southern Hemisphere - Max Temp vs. Latitude Linear Regression

In [ ]:
x_values = southern_df["Lat"]
y_values = southern_df["Max Temp"]

(slope, intercept, rvalue, pvalue, stderr) = linregress(x_values, y_values)
regress_values = x_values * slope + intercept
line_eq = "y = " + str(round(slope,2)) + "x + " + str(round(intercept,2))

plt.scatter(x_values, y_values, marker="o", facecolors="lightblue", edgecolors="black")

plt.plot(x_values,regress_values,"r-")

plt.annotate(line_eq,(-30,275),fontsize=15,color="red")

#Add title and other labels
plt.title("Southern Hemisphere: City Latitude vs. Max Temperature (07/29/2020)")
plt.xlabel("Latitude")
plt.ylabel("Max Temperature (F)")

# Print out the r-squared value along with the plot.
print(f"The r-squared is: {rvalue**2}")

# Show and save plot
plt.savefig("output_data/South_Lat_Temp_Plot.png")
plt.show()

# Northern Hemisphere - Humidity (%) vs. Latitude Linear Regression

In [ ]:
x_values = northern_df["Lat"]
y_values = northern_df["Humidity"]

(slope, intercept, rvalue, pvalue, stderr) = linregress(x_values, y_values)
regress_values = x_values * slope + intercept
line_eq = "y = " + str(round(slope,2)) + "x + " + str(round(intercept,2))

plt.scatter(x_values, y_values, marker="o", facecolors="lightblue", edgecolors="black")

plt.plot(x_values,regress_values,"r-")

plt.annotate(line_eq,(45,15),fontsize=15,color="red")

#Add title and other labels
plt.title("Northern Hemisphere: City Latitude vs. Humidity (07/29/2020)")
plt.xlabel("Latitude")
plt.ylabel("Humidity (%)")

# Print out the r-squared value along with the plot.
print(f"The r-squared is: {rvalue**2}")

# Show and save plot
plt.savefig("output_data/North_Lat_Humidity_Plot.png")
plt.show()

# Southern Hemisphere - Humidity (%) vs. Latitude Linear Regression

In [ ]:
x_values = southern_df["Lat"]
y_values = southern_df["Humidity"]

(slope, intercept, rvalue, pvalue, stderr) = linregress(x_values, y_values)
regress_values = x_values * slope + intercept
line_eq = "y = " + str(round(slope,2)) + "x + " + str(round(intercept,2))

plt.scatter(x_values, y_values, marker="o", facecolors="lightblue", edgecolors="black")

plt.plot(x_values,regress_values,"r-")

plt.annotate(line_eq,(-55,30),fontsize=15,color="red")

#Add title and other labels
plt.title("Southern Hemisphere: City Latitude vs. Humidity (07/29/2020)")
plt.xlabel("Latitude")
plt.ylabel("Humidity (%)")

# Print out the r-squared value along with the plot.
print(f"The r-squared is: {rvalue**2}")

# Show and save plot
plt.savefig("output_data/South_Lat_Humidity_Plot.png")
plt.show()

# Northern Hemisphere - Cloudiness (%) vs. Latitude Linear Regression

In [ ]:
x_values = northern_df["Lat"]
y_values = northern_df["Cloudiness"]

(slope, intercept, rvalue, pvalue, stderr) = linregress(x_values, y_values)
regress_values = x_values * slope + intercept
line_eq = "y = " + str(round(slope,2)) + "x + " + str(round(intercept,2))

plt.scatter(x_values, y_values, marker="o", facecolors="lightblue", edgecolors="black")

plt.plot(x_values,regress_values,"r-")

plt.annotate(line_eq,(45,30),fontsize=15,color="red")

#Add title and other labels
plt.title("Northern Hemisphere: City Latitude vs. Cloudiness (07/29/2020)")
plt.xlabel("Latitude")
plt.ylabel("Cloudiness (%)")

# Print out the r-squared value along with the plot.
print(f"The r-squared is: {rvalue**2}")

# Show and save plot
plt.savefig("output_data/North_Lat_Cloudiness_Plot.png")
plt.show()

# Southern Hemisphere - Cloudiness (%) vs. Latitude Linear Regression

In [ ]:
x_values = southern_df["Lat"]
y_values = southern_df["Cloudiness"]

(slope, intercept, rvalue, pvalue, stderr) = linregress(x_values, y_values)
regress_values = x_values * slope + intercept
line_eq = "y = " + str(round(slope,2)) + "x + " + str(round(intercept,2))

plt.scatter(x_values, y_values, marker="o", facecolors="lightblue", edgecolors="black")

plt.plot(x_values,regress_values,"r-")

plt.annotate(line_eq,(-50,30),fontsize=15,color="red")

#Add title and other labels
plt.title("Southern Hemisphere: City Latitude vs. Cloudiness (07/29/2020)")
plt.xlabel("Latitude")
plt.ylabel("Cloudiness (%)")

# Print out the r-squared value along with the plot.
print(f"The r-squared is: {rvalue**2}")

# Show and save plot
plt.savefig("output_data/South_Lat_Cloudiness_Plot.png")
plt.show()

# Northern Hemisphere - Wind Speed (mph) vs. Latitude Linear Regression

In [ ]:
x_values = northern_df["Lat"]
y_values = northern_df["Wind Speed"]

(slope, intercept, rvalue, pvalue, stderr) = linregress(x_values, y_values)
regress_values = x_values * slope + intercept
line_eq = "y = " + str(round(slope,2)) + "x + " + str(round(intercept,2))

plt.scatter(x_values, y_values, marker="o", facecolors="lightblue", edgecolors="black")

plt.plot(x_values,regress_values,"r-")

plt.annotate(line_eq,(0,11),fontsize=15,color="red")

#Add title and other labels
plt.title("Southern Hemisphere: City Latitude vs. Wind Speed (07/29/2020)")
plt.xlabel("Latitude")
plt.ylabel("Wind Speed (mph)")

# Print out the r-squared value along with the plot.
print(f"The r-squared is: {rvalue**2}")

# Show and save plot
plt.savefig("output_data/North_Lat_Wind_Plot.png")
plt.show()

# Southern Hemisphere - Wind Speed (mph) vs. Latitude Linear Regression

In [ ]:
x_values = southern_df["Lat"]
y_values = southern_df["Wind Speed"]

(slope, intercept, rvalue, pvalue, stderr) = linregress(x_values, y_values)
regress_values = x_values * slope + intercept
line_eq = "y = " + str(round(slope,2)) + "x + " + str(round(intercept,2))

plt.scatter(x_values, y_values, marker="o", facecolors="lightblue", edgecolors="black")

plt.plot(x_values,regress_values,"r-")

plt.annotate(line_eq,(-50,8),fontsize=15,color="red")

#Add title and other labels
plt.title("Southern Hemisphere: City Latitude vs. Wind Speed (07/29/2020)")
plt.xlabel("Latitude")
plt.ylabel("Wind Speed (mph)")

# Print out the r-squared value along with the plot.
print(f"The r-squared is: {rvalue**2}")

# Show and save plot
plt.savefig("output_data/South_Lat_Wind_Plot.png")
plt.show()